In [1]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np

In [2]:
uri = "bolt://34.101.57.12:7687"
username = "neo4j"
password = "unej1234"

driver = GraphDatabase.driver(uri, auth=(username, password))

In [3]:
def extract_node_properties():
    with driver.session() as session:
        # Cypher query to fetch node properties
        query = """
        MATCH (n)
        RETURN n.Vector AS vector, n.label AS label, labels(n) AS kelas, n.abstract AS keterangan
        """
        result = session.run(query)
        # Extract properties and store in DataFrame
        df = pd.DataFrame([record.values() for record in result], columns=result.keys())
        return df


In [4]:
node_df = extract_node_properties()
cleaned_kelas = node_df['kelas'].apply(lambda entry: [item for item in entry if item not in ["Resource", "NamedIndividual"]])
node_df['kelas'] = cleaned_kelas
node_df

,vector,label,kelas,keterangan
0,"[-0.48007139563560486, 0.6460448503494263, -0....",Metalaxyl,[Fungisida],Metalaxyl adalah sejenis fungisida yang diguna...
1,"[0.26170042157173157, 0.07159162312746048, 0.0...",Gosong bulir,[PenyakitPadi],"Penyakit gosong bulir padi, juga dikenal sebag..."
2,"[0.022821197286248207, 0.3413693904876709, -0....",Acidovorax avenae subsp. avenae,[PatogenPadi],Acidovorax avenae subsp. avenae adalah bakteri...
3,"[-0.0965781956911087, 0.03939099982380867, -0....",Tanaman layu,[Gejala],Gejala layu pada tanaman padi adalah tanda yan...
4,"[0.39404162764549255, -0.3328622579574585, 0.4...",Bibit mati,[Gejala],Gejala kematian bibit padi dapat terlihat dala...
...,...,...,...,...
100,"[0.6989031434059143, -0.3856666088104248, 0.55...",Bacillus thuringiensis var. israelensis,[Pestisida],Bacillus thuringiensis var. israelensis (Bti) ...
101,"[0.4118303656578064, -0.5445472598075867, -0.1...",Belalang Sawah,[HamaPadi],"Belalang sawah, atau Nilaparvata lugens, adala..."
102,"[-0.4613795280456543, 0.18603742122650146, 0.3...",Bakteri Garis Daun,[PenyakitPadi],Penyakit bakteri garis daun pada padi disebabk...
103,"[-0.24614253640174866, 0.1480824500322342, -0....",Fusarium spp.,[PatogenPadi],Fusarium spp. adalah kelompok jamur patogen ya...


In [5]:
def extract_node_properties_and_relationships():
    with driver.session() as session:
        # Cypher query to fetch node properties and relationships
        query = """
        MATCH (n)-[r]->(m)
        RETURN n.Vector AS source_vector, n.label AS source_label, labels(n) AS source_class, 
               type(r) AS relationship_type,
               m.Vector AS target_vector, m.label AS target_label, labels(m) AS target_class
        """
        result = session.run(query)
        # Extract properties and relationships and store in DataFrame
        df = pd.DataFrame([record.values() for record in result], columns=result.keys())
        # Clean labels
        df['source_class'] = df['source_class'].apply(lambda entry: [item for item in entry if item not in ["Resource", "NamedIndividual"]])
        df['target_class'] = df['target_class'].apply(lambda entry: [item for item in entry if item not in ["Resource", "NamedIndividual"]])
        return df

# Call the function to extract node properties and relationships
Rice_KG_df = extract_node_properties_and_relationships()

# Print the DataFrame
Rice_KG_df


,source_vector,source_label,source_class,relationship_type,target_vector,target_label,target_class
0,"[0.26170042157173157, 0.07159162312746048, 0.0...",Gosong bulir,[PenyakitPadi],diberikanFungisida,"[-0.0959160327911377, 0.14652732014656067, 0.0...",Pyraclostrobin,[Fungisida]
1,"[0.26170042157173157, 0.07159162312746048, 0.0...",Gosong bulir,[PenyakitPadi],memilikiGejala,"[0.10879369080066681, 0.07831495255231857, -0....",Bulir mengalami kerusakan,[Gejala]
2,"[0.26170042157173157, 0.07159162312746048, 0.0...",Gosong bulir,[PenyakitPadi],memilikiGejala,"[0.6842259168624878, -0.09251340478658676, 0.0...",Bulir pecah,[Gejala]
3,"[0.26170042157173157, 0.07159162312746048, 0.0...",Gosong bulir,[PenyakitPadi],terkenaPatogen,"[0.24190634489059448, 0.04473293945193291, 0.0...",Tilletia barclayana,[PatogenPadi]
4,"[0.26170042157173157, 0.07159162312746048, 0.0...",Gosong bulir,[PenyakitPadi],memilikiGejala,"[0.2921145260334015, -0.051839686930179596, -0...",Bulir terdapat bercak,[Gejala]
...,...,...,...,...,...,...,...
142,"[-0.4613795280456543, 0.18603742122650146, 0.3...",Bakteri Garis Daun,[PenyakitPadi],memilikiGejala,"[-0.3528200387954712, -0.11492333561182022, 0....",Daun mengering,[Gejala]
143,"[-0.4613795280456543, 0.18603742122650146, 0.3...",Bakteri Garis Daun,[PenyakitPadi],diberikanBakterisida,"[-0.43456384539604187, 0.053754474967718124, 0...",Fosfit,[Bakterisida]
144,"[-0.4613795280456543, 0.18603742122650146, 0.3...",Bakteri Garis Daun,[PenyakitPadi],diberikanBakterisida,"[-0.5201900601387024, -0.21731899678707123, -0...",Streptomisin,[Bakterisida]
145,"[-0.4613795280456543, 0.18603742122650146, 0.3...",Bakteri Garis Daun,[PenyakitPadi],memilikiGejala,"[-0.8089390993118286, -0.32538989186286926, 0....",Malai mengalami kerusakan,[Gejala]


In [65]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Concatenate source and target vectors as features
X = np.concatenate([Rice_KG_df['source_vector'].values.tolist(), Rice_KG_df['target_vector'].values.tolist()], axis=1)

# Encode categorical variables
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(Rice_KG_df['relationship_type'])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.7111111111111111


In [56]:
from sklearn.tree import DecisionTreeClassifier

# Encode categorical variables if needed
label_encoder = LabelEncoder()
X_encoded = X.copy()  # Make a copy of X to avoid modifying the original data
for i in range(X_encoded.shape[1]):  # Iterate over columns (features)
    X_encoded[:, i] = label_encoder.fit_transform(X_encoded[:, i])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

# Train decision tree classifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

# Make predictions
y_pred = classifier.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.7333333333333333


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Concatenate source and target vectors as features
X = np.concatenate([Rice_KG_df['source_vector'].values.tolist(), Rice_KG_df['target_vector'].values.tolist()], axis=1)

# Encode categorical variables
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(Rice_KG_df['relationship_type'])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax') # Output layer with softmax activation for multi-class classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', # Use sparse categorical crossentropy for integer-encoded labels
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)


Epoch 1/10


c:\Users\Ariful\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0704 - loss: 2.1270  
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1621 - loss: 2.0356 
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 427us/step - accuracy: 0.2664 - loss: 1.9600
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3463 - loss: 1.9007 
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.4424 - loss: 1.8257  
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.4395 - loss: 1.7722  
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4520 - loss: 1.6894 
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4270 - loss: 1.6732 
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.4770 - loss: 1.5812  
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4520 - loss: 1.5512 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.3667 - loss: 1.6186
Accuracy: 0.36666667461395264


In [21]:
X

array([[ 0.26170042,  0.07159162,  0.04994822, ..., -0.76906621,
        -0.39284524, -1.1442076 ],
       [ 0.26170042,  0.07159162,  0.04994822, ..., -1.15756714,
        -0.00530381, -0.90904421],
       [ 0.26170042,  0.07159162,  0.04994822, ..., -1.4618758 ,
        -0.05329949, -1.02885365],
       ...,
       [-0.46137953,  0.18603742,  0.37038094, ..., -0.98699993,
        -0.68396753, -0.44398031],
       [-0.46137953,  0.18603742,  0.37038094, ..., -0.79979306,
        -0.15046796, -0.58907413],
       [-0.46137953,  0.18603742,  0.37038094, ..., -1.35491252,
        -0.50807852, -0.15630077]])